In [ ]:
# metric length scale

# need: inital length in pixels (w), initial height (h_i), height at time t (h_t)
# w = 476 pixels
# CHECK INITIAL HEIGHT OF 12:15PM TRIAL; divide feet by 3.28084 to get measurements in metres
# spacing(t) = (h_i/h_t)*w


In [ ]:
# generate displacements

# potential errors: no positions 

pos_keys = dictionary.keys(cTracker.Positions)

def get_Displacements(dictionary):
    if not dictionary:
        print("The dictionary is empty.")
    Displacements = OrderedDict()
    for k in pos_keys:
        T = len(dictionary[k][:,0])
        for t in range(T):
            if t == 0:
                u_x = 0
                u_y = 0
                u_mag = 0
                new_items = np.array([t, u_x, u_y, u_mag])
                Displacements[k] = new_items 
            else:
                u_x = dictionary[k][t, 1] - dictionary[k][t-1, 1]
                u_y = dictionary[k][t, 2] - dictionary[k][t-1, 2]
                u_mag = np.sqrt(u_x**2 + u_y**2)
                new_items = np.array([t, u_x, u_y, u_mag])
                Displacements[k] = np.append(Displacements[k], new_items, axis=0)
    
    return Displacements

    # find vMaxSheep


In [ ]:
# get observables

from shapely.geometry import Polygon

# Pressure

# at each frame frameNumber, we want to extract the positions of each agent and add them to the generator array. 
# we then want to create a voronoi diagram, calculate the areas of the finite cells, and determine the pressure.

disp_keys = dictionary.keys(Displacements)

def get_Pressure(dictionary):
    generators = np.array([[]])
    areas = np.array([])
    pressures = np.array([[]])
    frameNumber=0
    while True:
        for k in pos_keys:
            if frameNumber in dictionary[k][:,0]:
                generators = np.append(generators, np.array([dictionary[k][frameNumber][1], dictionary[k][frameNumber][2]]), axis=0)
        N = len(generators) #check this
        if N==0:
            break
        vor = Voronoi(generators)
        finite_regions = [region for region in vor.regions if not -1 in region and len(region) > 0]
        region_polygons = []
        for region_indices in finite_regions:
            polygon_vertices = vor.vertices[region_indices]
            region_polygons.append(polygon_vertices)
    
        for poly_vertices in region_polygons:
            polygon = Polygon(poly_vertices)
            areas = np.append(areas, polygon.area)

        p_t = (1/N)*np.sum(np.reciprocal(areas))
        
        pressures = np.append(pressures, np.array[frameNumber, p_t])
        frameNumber +=1
        
    
# Polarization

# at each frameNumber, we want to extract the displacements of each agent and vector-sum them. 
# we then want to take the vector norm and divide out by the number of sheep

def get_Polarizations(dictionary): 
    disp_t = []
    polarizations = []
    # runDuration = ???
    for frameNumber in runDuration:
        for k in disp_keys:
            if frameNumber in dictionary[k][:,0]:
                disp_t = np.append(disp_t, np.array[dictionary[k][frameNumber, 1], dictionary[k][frameNumber, 2]])
        N = len(disp_t)
        disp_sum_t = np.sum(disp_t, axis=0)
        polarization_t = (1/N)*np.linalg.norm(disp_sum_t)
        polarizations = np.append(polarizations, np.array[frameNumber, polarization_t])

# Mixing

In [ ]:
# find L in metres